In [1]:
library(SnapATAC)
library(Seurat)
library(Signac)
library(genomation)
library(GenomicRanges)
library(parallel)
library(foreach)

Loading required package: Matrix

Loading required package: rhdf5

Warning message:
“no DISPLAY variable so Tk is not available”
Loading required package: SeuratObject

Loading required package: sp

‘SeuratObject’ was built under R 4.3.2 but the current version is
4.3.3; it is recomended that you reinstall ‘SeuratObject’ as the ABI
for R may have changed

‘SeuratObject’ was built with package ‘Matrix’ 1.6.3 but the current
version is 1.6.5; it is recomended that you reinstall ‘SeuratObject’ as
the ABI for ‘Matrix’ may have changed


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect


Loading required package: grid

Warning message:
“replacing previous import ‘Biostrings::pattern’ by ‘grid::pattern’ when loading ‘genomation’”
Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following object is masked from ‘package:SeuratObject’:

    intersect


The following objects ar

Specify file path

In [2]:
gene_gtf_path = "/maps/projects/ralab/data/genome/hg38/gencode.v43.chr_patch_hapl_scaff.annotation.gtf"
abc_genes_path = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/241203/scE2G/ENCODE_rE2G/ABC/reference/hg38/CollapsedGeneBounds.hg38.TSS500bp.bed"
path.pairs.E2G = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/240508/sc-E2G/test/results_K562_Xu/K562/Kendall/Pairs.tsv.gz"
path.pairs.ABC = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/240508/sc-E2G/test/results_K562_Xu/K562/Predictions/EnhancerPredictionsAllPutative.tsv.gz"
path.matrix.atac_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/240508/sc-E2G/test/results_K562_Xu/K562/Kendall/atac_matrix.csv.gz"
path.matrix.rna_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/K562_Xu/1.prepare_data/1.seurat_pipeline.240507/rna_count_matrix.csv.gz"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/K562_Xu/3.Genome_wide_prediction/SnapATAC/SnapATAC.250212/"

In [3]:
n.cores = 8

Import candidate E-G pairs

In [4]:
pairs.E2G = readGeneric(path.pairs.E2G,
                        header = T,
                        keep.all.metadata = T)

Import ABC results

In [5]:
pairs.ABC = readGeneric(path.pairs.ABC,
                        header = T,
                        keep.all.metadata = T)

Filter ABC results which distance < 1M

In [6]:
pairs.ABC.1M = pairs.ABC[pairs.ABC$distance < 10^6]

Filter E-G pairs overlaping with pairs.ABC.1M

In [7]:
df.pairs.E2G.chr_rename = as.data.frame(pairs.E2G)[,1:3]
df.pairs.E2G.chr_rename[,"seqnames"] = paste(seqnames(pairs.E2G),
                                             mcols(pairs.E2G)[,"TargetGene"],
                                             sep = "_")
pairs.E2G.chr_rename = GRanges(df.pairs.E2G.chr_rename)
rm(df.pairs.E2G.chr_rename)

df.pairs.ABC.1M.chr_rename = as.data.frame(pairs.ABC.1M)[,1:3]
df.pairs.ABC.1M.chr_rename[,"seqnames"] = paste(seqnames(pairs.ABC.1M),
                                                mcols(pairs.ABC.1M)[,"TargetGene"],
                                                sep = "_")
pairs.ABC.1M.chr_rename = GRanges(df.pairs.ABC.1M.chr_rename)
rm(df.pairs.ABC.1M.chr_rename)

pairs.E2G.filter = pairs.E2G[countOverlaps(pairs.E2G.chr_rename,
                                           pairs.ABC.1M.chr_rename) > 0]
rm(pairs.E2G.chr_rename)
rm(pairs.ABC.1M.chr_rename)
pairs.E2G.filter

GRanges object with 3150487 ranges and 3 metadata columns:
            seqnames            ranges strand |  TargetGene
               <Rle>         <IRanges>  <Rle> | <character>
        [1]     chr1 10001196-10001745      * | APITD1-CORT
        [2]     chr1 10001196-10001745      * |       CASZ1
        [3]     chr1 10001196-10001745      * |       CENPS
        [4]     chr1 10001196-10001745      * |      CLSTN1
        [5]     chr1 10001196-10001745      * |        CORT
        ...      ...               ...    ... .         ...
  [3150483]     chrX     989754-991253      * |     SLC25A6
  [3150484]     chrX   9995827-9996208      * |       CLCN4
  [3150485]     chrX   9995827-9996208      * |       FAM9B
  [3150486]     chrX   9995827-9996208      * |       TBL1X
  [3150487]     chrX   9995827-9996208      * |        WWC3
                          PeakName               PairName
                       <character>            <character>
        [1] chr1-10001196-10001745 chr1-10001

Import ATAC matrix

In [8]:
matrix.atac_count = read.csv(path.matrix.atac_count,
                             row.names = 1,
                             check.names = F)
matrix.atac_count = Matrix(as.matrix(matrix.atac_count), sparse = TRUE)
matrix.atac = BinarizeCounts(matrix.atac_count)
rm(matrix.atac_count)

Import RNA matrix

In [9]:
matrix.rna_count = read.csv(path.matrix.rna_count,
                            row.names = 1,
                            check.names = F)
matrix.rna_count = Matrix(as.matrix(matrix.rna_count), sparse = TRUE)
matrix.rna_count = matrix.rna_count[,colnames(matrix.atac)]
matrix.rna_count = matrix.rna_count[rowSums(matrix.rna_count) > 0,]
matrix.rna = NormalizeData(matrix.rna_count)
rm(matrix.rna_count)

Map gene names

In [10]:
extract_attributes <- function(gtf_attributes, att_of_interest){
  att <- unlist(strsplit(gtf_attributes, " "))
  if(att_of_interest %in% att){
    return(gsub("\"|;","", att[which(att %in% att_of_interest)+1]))
  } else {
    return(NA)}
}
map_gene_names <- function(rna_matrix, gene_gtf_path, abc_genes_path){
    library(dplyr)
    library(data.table)
    
	gene_ref <- fread(gene_gtf_path, header = FALSE, sep = "\t") %>%
		setNames(c("chr","source","type","start","end","score","strand","phase","attributes")) %>%
		dplyr::filter(type == "gene")
	gene_ref$gene_ref_name <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_name"))
	gene_ref$Ensembl_ID <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_id"))
	gene_ref <- dplyr::select(gene_ref, gene_ref_name, Ensembl_ID) %>%
		mutate(Ensembl_ID = sub("\\.\\d+$", "", Ensembl_ID)) %>% # remove decimal digits 
		distinct()
	
	abc_genes <- fread(abc_genes_path, col.names = c("chr", "start", "end", "name", "score", "strand", "Ensembl_ID", "gene_type")) %>%
		dplyr::select(name, Ensembl_ID) %>%
		rename(abc_name = name) %>%
		left_join(gene_ref, by = "Ensembl_ID") %>%
		group_by(Ensembl_ID) %>% # remove cases where multiple genes map to one ensembl ID
		filter(n() == 1) %>%
		ungroup()

	gene_key <- abc_genes$abc_name
	names(gene_key) <- abc_genes$gene_ref_name

	# remove genes not in our gene universe	
	row_sub <- intersect(rownames(rna_matrix), names(gene_key)) # gene ref names
	rna_matrix_filt <- rna_matrix[row_sub,] # still gene ref names
	rownames(rna_matrix_filt) <- gene_key[row_sub] # converted to abc names

	return(rna_matrix_filt)
}

In [11]:
matrix.rna.rename = map_gene_names(matrix.rna,gene_gtf_path, abc_genes_path)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:GenomicRanges’:

    intersect, setdiff, union


The following object is masked from ‘package:GenomeInfoDb’:

    intersect


The following objects are masked from ‘package:IRanges’:

    collapse, desc, intersect, setdiff, slice, union


The following objects are masked from ‘package:S4Vectors’:

    first, intersect, rename, setdiff, setequal, union


The following objects are masked from ‘package:BiocGenerics’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:GenomicRanges’:

    shift


The following object is masked from ‘package:IRanges’:

    shift


The following objects are masked

In [12]:
pairs.E2G.filter2 = pairs.E2G.filter[pairs.E2G.filter$TargetGene %in% rownames(matrix.rna.rename) &
                                     pairs.E2G.filter$PeakName %in% rownames(matrix.atac)]

Run SnapATAC prediction

In [13]:
PredictionSnapATAC = function(pairs.E2G,
                              data.RNA,
                              data.ATAC,
                              n.core = 16){
  my.cluster <- parallel::makeCluster(
    n.core,
    type = "PSOCK"
  )
  doParallel::registerDoParallel(cl = my.cluster)
  
  pairs.E2G =  
    foreach (index.tmp = 1:length(pairs.E2G),
             .combine = 'c',
             .packages = c("Matrix",
                           "GenomicRanges")) %dopar% {

                             pairs.E2G.tmp = pairs.E2G[index.tmp]
                             
                             model.tmp = summary(stats::glm(data.ATAC[pairs.E2G.tmp$PeakName,] ~ data.RNA[pairs.E2G.tmp$TargetGene,], 
                                                            family = binomial(link = "logit")))[["coefficients"]]
                             mcols(pairs.E2G.tmp)[,c("beta","sd","z","p")] = 
                               t(data.frame(model.tmp[2,]))
                             
                             pairs.E2G.tmp
                           }
  
  parallel::stopCluster(cl = my.cluster)
  
  return(pairs.E2G)
}

In [14]:
dir.create(paste(dir.output,"chr",sep = "/"),recursive = T)

In [15]:
chr.done = dir(paste(dir.output,"chr",sep = "/"))
chr.run = as.character(unique(seqnames(pairs.E2G.filter2)))
chr.run = chr.run[!chr.run %in% chr.done]

In [16]:
for(chr.tmp in chr.run){
  print(chr.tmp)
  pairs.E2G.chr.res = pairs.E2G.filter2[seqnames(pairs.E2G.filter2) == chr.tmp]
  mcols(pairs.E2G.chr.res)[,c("beta","sd","z","p")] = NA
  pairs.E2G.chr.res = PredictionSnapATAC(pairs.E2G.chr.res,
                                         matrix.rna.rename[rownames(matrix.rna.rename) %in% pairs.E2G.chr.res$TargetGene,],
                                         matrix.atac[rownames(matrix.atac) %in% pairs.E2G.chr.res$PeakName,],
                                         n.cores)
  saveRDS(pairs.E2G.chr.res,
          paste(dir.output,"chr",chr.tmp,sep = "/"))
  rm (pairs.E2G.chr.res)
}

[1] "chr1"
[1] "chr10"
[1] "chr11"
[1] "chr12"
[1] "chr13"
[1] "chr14"
[1] "chr15"
[1] "chr16"
[1] "chr17"
[1] "chr18"
[1] "chr19"
[1] "chr2"
[1] "chr20"
[1] "chr21"
[1] "chr22"
[1] "chr3"
[1] "chr4"
[1] "chr5"
[1] "chr6"
[1] "chr7"
[1] "chr8"
[1] "chr9"
[1] "chrX"


Merge results

In [17]:
chr.merge = as.character(unique(seqnames(pairs.E2G.filter2)))
list.res = list()
for(chr.tmp in chr.merge){
    list.res[[chr.tmp]] = readRDS(paste(dir.output,"chr",chr.tmp,sep = "/"))
}
pairs.E2G.res = unlist(as(list.res, "GRangesList"))

Save results

In [18]:
saveRDS(pairs.E2G.res,
        paste(dir.output,"pairs.E2G.res.rds",sep = "/"))
pairs.E2G.res

GRanges object with 3046227 ranges and 7 metadata columns:
       seqnames            ranges strand |  TargetGene               PeakName
          <Rle>         <IRanges>  <Rle> | <character>            <character>
  chr1     chr1 10001196-10001745      * | APITD1-CORT chr1-10001196-10001745
  chr1     chr1 10001196-10001745      * |       CASZ1 chr1-10001196-10001745
  chr1     chr1 10001196-10001745      * |       CENPS chr1-10001196-10001745
  chr1     chr1 10001196-10001745      * |      CLSTN1 chr1-10001196-10001745
  chr1     chr1 10001196-10001745      * |        CORT chr1-10001196-10001745
   ...      ...               ...    ... .         ...                    ...
  chrX     chrX     989754-991253      * |     SLC25A6     chrX-989754-991253
  chrX     chrX   9995827-9996208      * |       CLCN4   chrX-9995827-9996208
  chrX     chrX   9995827-9996208      * |       FAM9B   chrX-9995827-9996208
  chrX     chrX   9995827-9996208      * |       TBL1X   chrX-9995827-9996208
  chr

In [19]:
df.output = as.data.frame(pairs.E2G.res,row.names = NULL)
colnames(df.output)[1] = "chr"
df.output[,"CellType"] = "K562"
df.output = df.output[,c("chr",
                         "start",
                         "end",
                         "TargetGene",
                         "CellType",
                         "beta",
                         "sd",
                         "z",
                         "p")]
data.table::fwrite(df.output,
                   file = paste(dir.output,"pairs.E2G.res.tsv.gz",sep = "/"),
                   row.names = F,
                   quote = F,
                   sep = "\t")
df.output

chr,start,end,TargetGene,CellType,beta,sd,z,p
<fct>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
chr1,10001196,10001745,APITD1-CORT,K562,0.96367698,1.0201033,0.94468569,0.3448193640
chr1,10001196,10001745,CASZ1,K562,1.45613277,1.1654277,1.24944063,0.2115039554
chr1,10001196,10001745,CENPS,K562,0.24709586,0.3898082,0.63389079,0.5261520982
chr1,10001196,10001745,CLSTN1,K562,0.19684217,0.2909205,0.67661845,0.4986480717
chr1,10001196,10001745,CORT,K562,-92.39012771,3762.1673621,-0.02455769,0.9804077702
chr1,10001196,10001745,CTNNBIP1,K562,0.05705119,0.2986616,0.19102282,0.8485077169
chr1,10001196,10001745,DFFA,K562,0.16739230,0.1958141,0.85485302,0.3926325204
chr1,10001196,10001745,GPR157,K562,-41.77387807,1439.2644515,-0.02902446,0.9768450800
chr1,10001196,10001745,H6PD,K562,0.01751790,0.6373034,0.02748754,0.9780708780


In [20]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.10 (Ootpa)

Matrix products: default
BLAS/LAPACK: /maps/projects/ralab/people/lpm537/software/anaconda3/envs/Notebook_E2G_240505/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Copenhagen
tzcode source: system (glibc)

attached base packages:
 [1] parallel  stats4    grid      stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] data.table_1.15.2    dplyr_1.1.4          foreach_1.5.2       
 [4] GenomicRanges_1.54.1 GenomeInfoDb_1.38.1  IRanges